# 오차역전파법

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# 단순한 계층 구현하기

곱셈 계층 먼저 구현 

- 순전파 : 입력을 *

- 역전파 : 상류에서 넘어온 미분 값에 순전파 값을 서로 바꾸고 *

In [18]:
# 곱셈 계층
class MulLayer(object):
    def __init__(self): # 인스턴스 변수 x 와 y 를 초기화
        self.x = None
        self.y = None
        
    def forward(self, x, y): # 순전파 시 인스턴스 변수 x와 y에 값이 들어감.
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, dout): # 역전파
        dx = dout * self.y # 상류의 값에 x와 y를 바꿔서 곱한다.
        dy = dout * self.x

        return dx, dy

In [19]:
# 곱셈 계층의 순전파
apple = 100
apple_num = 2
tax = 1.1

# 계층들을 객체로
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num) # 순수 사과 가격
price = mul_tax_layer.forward(apple_price, tax) # 세금 포함 가격

print(apple_price)
print(price)

200
220.00000000000003


In [20]:
# 역전파
dprice = 1 # cost에 대한 처음 미분 값 당연히 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
print(dapple_price, dtax)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(dapple, dapple_num)

1.1 200
2.2 110.00000000000001


덧셈 계층 구현

- 순전파 : 그냥 더해주면 끝
- 역전파 : 상류에서 온 미분 값 그대로 반환하면 끝

In [21]:
# 덧셈 계층 구현
class AddLayer(object):
    def __init__(self): # 덧셈 계층은 저장해놓고 쓸 인스턴스 변수가 없다.
        pass
    
    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout*1 # 반환은 두개 해줘야 함
        dy = dout*1
        return dx, dy

In [22]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num) #(1)
orange_price = mul_orange_layer.forward(orange, orange_num) #(2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)#(3)
price = mul_tax_layer.forward(all_price,tax)#(4)
print("< 순전파 결과(총 가격) >")
print(price)

print("< 역전파 결과 순서대로 >")
# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) #(4)
print(dall_price, dtax)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) #(3)
print(dapple_price, dorange_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) #(2)
print(dorange, dorange_num)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) #(1)
print(dapple, dapple_num)

< 순전파 결과(총 가격) >
715.0000000000001
< 역전파 결과 순서대로 >
1.1 650
1.1 1.1
3.3000000000000003 165.0
2.2 110.00000000000001


# 활성화 함수 계층 구현하기

ReLU 계층

ReLU 수식

$y = \begin{cases}x&(x > 0)\\0&(x \le 0)\end{cases}​$

x에 대한 y의 미분

$\frac{\partial{y}}{\partial{x}} = \begin{cases}1&(x > 0)\\0&(x \le 0)\end{cases}​$

In [23]:
class Relu(object):
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0) # 0이하의 값만 True
        out = x.copy() # 입력을 그대로 받아서
        out[self.mask] = 0 # 0이하의 값은 0으로 바꿔준다

        return out

    def backward(sefl,dout): # 0보다 크면 상류 흐름 그대로 0보다 이하면 0
        dout[self.mask] = 0 # 0이하의 값만 0으로 바꿔준다
        dx = dout 

        return dx

In [24]:
x = np.array( [[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [25]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


In [26]:
out = x.copy()
print(out[mask])

out[mask] = 0 # True에 해당하는 원소만 0으로 바꿈
print(out) # 0 이하의 값은 0으로 변경됐음

[-0.5 -2. ]
[[1. 0.]
 [0. 3.]]


Sigmoid 계층

$y = \frac{1}{1+e^{-x}}$

x에 대해 y를 미분하면

$\frac{\partial{y}}{\partial{x}} = y(1-y)$

In [27]:
class Sigmoid(object):
    def __init__(self):
        self.out = None # 인스턴스 변수 Out (위 식에서 y에 해당)

    def forward(self, x):
        out = 1 / (1 + np.exp(-x)) # 그냥 순전파
        self.out = out # 역전파때 사용할 y

        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out # y를 그대로 다시 사용

        return dx

# Affine / Softmax 계층 구현하기

In [28]:
class Affine(object):
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None # 순전파시 입력 x를 담아 둘 인스턴스 변수
        self.dW = None # 역전파시 Loss를 W로 편미분한 값
        self.db = None # 역전파시 Loss를 b로 편미분한 값

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self,dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout) # X_transpose * dout
        self.db = np.sum(dout, axis = 0) 

        return dx

In [34]:
def softmax(a): 
    c = np.max(a) 
    exp_a = np.exp(a) 
    sum_exp_a = np.sum(exp_a) 
    y = exp_a / sum_exp_a 
    return y

class SoftmaxWithLoss(object):
    def __init__(self):
        self.loss = None # 손실
        self.y = None # softmax의 출력
        self.t = None # 정답 레이블 (one-hot-vector)

    
    def forward(self, x, t):
        self.t = t # 역전파 때 사용해야 하므로 인스턴스 변수에 저장
        self.y = softmax(x) 
        self.loss = cross_entropy_error(self.y, self.t) 
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0] # batch의 크기
        dx = (self.y - self.t) / batch_size 
        
        return dx